In [3]:
import numpy as np

#drawing 2D image.
#import matplotlib.pyplot as mpt

import pandas as pd

In [ ]:
data_set=pd.read_csv('../venv/Datasets/DDL_DDoS10_CSV')

x1=data_set.iloc[:,:-1].values  #all data except the last colume
#x2=data_set.iloc[:,6]

In [35]:
# extracting partial data; 
# ["No.","Time","Source","Destination","Protocol","Length","Info"]
x1=data_set.iloc[:,:-1].values  #all data except the last colume
x2=data_set.iloc[:,6]

In [36]:
#delete specific line. like the first line.


In [37]:
'''
# filling null value or missing value, could using mean.
# But non-numeric data can not use mean like Date or IP address which needs to be transformed.
#to be continued...  tranforming non-numeric data.
from sklearn.impute import SimpleImputer 
imputer= SimpleImputer(missing_values ='NaN', strategy='mean')  
#Fitting imputer object to the independent variables x.   
imputerimputer= imputer.fit(x1[:, 2:5])  
#Replacing missing data with the calculated mean value  
x1[:, 2:5]= imputer.transform(x1[:, 2:5])
'''

"\n# filling null value or missing value, could using mean.\n# But non-numeric data can not use mean like Date or IP address which needs to be transformed.\n#to be continued...  tranforming non-numeric data.\nfrom sklearn.impute import SimpleImputer \nimputer= SimpleImputer(missing_values ='NaN', strategy='mean')  \n#Fitting imputer object to the independent variables x.   \nimputerimputer= imputer.fit(x1[:, 2:5])  \n#Replacing missing data with the calculated mean value  \nx1[:, 2:5]= imputer.transform(x1[:, 2:5])\n"

In [38]:
#缺失值查看
print(data_set.isnull().sum()) 
#统计每列有几个缺失值
missing_col = data_set.columns[data_set.isnull().any()].tolist() 
#找出存在缺失值的列


#统计每个变量的缺失值占比def CountNA(data_set):
cols = data_set.columns.tolist() 
#cols为data的所有列名
n_df = data_set.shape[0] 
#n_df为数据的行数
for col in cols:missing = np.count_nonzero(data_set[col].isnull().values) 
#col列中存在的缺失值个数
mis_perc = float(missing) / n_df * 100
print("{col}的缺失比例是{miss}%".format(col=col,miss=mis_perc))


No.            0
Time           0
Source         0
Destination    0
Protocol       0
Length         0
Info           0
dtype: int64
Info的缺失比例是0.0%


In [ ]:
'''在数据量比较大时候或者一条记录中多个字段缺失，不方便填补的时候可以选择删除缺失值
data.dropna(axis=0,how="any",inplace=True)  #axis=0代表'行','any'代表任何空值行,若是'all'则代表所有值都为空时，才删除该行
data.dropna(axis=0,inplace=True) #删除带有空值的行
data.dropna(axis=1,inplace=True) #删除带有空值的列
'''
'''
 固定值填充
data = data.fillna(0) #缺失值全部用0插补data['col_name'] = data['col_name'].fillna('UNKNOWN') #某列缺失值用固定值插补
出现最频繁值填充,即众数插补，离散/连续数据都行，适用于名义变量，如性别

freq_port = data.col_name.dropna().mode()[0] # mode返回出现最多的数据,col_name为列名data['col_name'] = data['col_name'].fillna(freq_port) #采用出现最频繁的值插补

 中位数/均值插补
data['col_name'].fillna(data['col_name'].dropna().median(),inplace=True) #中位数插补，适用于偏态分布或者有离群点的分布data['col_name'].fillna(data['col_name'].dropna().mean(),inplace=True) #均值插补，适用于正态分布

用前后数据填充
data['col_name'] = data['col_name'].fillna(method='pad') #用前一个数据填充data['col_name'] = data['col_name'].fillna(method='bfill') #用后一个数据填充

拉格朗日插值法,一般针对有序的数据，如带有时间列的数据集,且缺失值为连续型数值小批量数据
from scipy.interpolate import lagrange#自定义列向量插值函数,s为列向量,n为被插值的位置,k为取前后的数据个数，默认5def ployinterp_columns(s, n, k=5):y = s[list(range(n-k,n)) + list(range(n+1,n+1+k))] #取数y = y[y.notnull()] #剔除空值return lagrange(y.index, list(y))(n) #插值并返回插值结果#逐个元素判断是否需要插值for i in data.columns:for j in range(len(data)):if (data[i].isnull())[j]: #如果为空即插值data[i][j] = ployinterp_columns(data[i],j)
'''

In [23]:
# Categorial data, further using virtual encoding.
#for type Variable  
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
from sklearn.compose import ColumnTransformer

# simple label type
label_encoder_x= LabelEncoder()  
x1[:, 5]= label_encoder_x.fit_transform(x1[:, 5])  

#Encoding for dummy variables  
transformer = ColumnTransformer(
    transformers=[
        ("OneHot",        # Just a name
         OneHotEncoder(), # The transformer class
         [4]              # The column(s) to be applied on.
         )
    ],
    remainder='passthrough' # donot apply anything to the remaining columns
)
x1 = transformer.fit_transform(x1)
#x1 = x1.astype('float64')   
#x1= onehot_encoder.fit_transform(x1).toarray()

IndexError: tuple index out of range

In [ ]:
'''
6.1 最小最大规范化

对原始数据进行线性变换，变换到[0,1]区间。计算公式为：x* = (x-x.min)/(x.max-x.min)

from sklearn.preprocessing import MinMaxScalerx_scaler = MinMaxScaler()y_scaler = MinMaxScaler()#特征归一化x_train_sca = x_scaler.fit_transform(X_train)x_test_sca = x_scaler.transform(X_test)y_train_sca = y_scaler.fit_transform(pd.DataFrame(y_train))

6.2 零均值规范化

对原始数据进行线性变换，经过处理的数据的均值为0，标准差为1。计算方式是将特征值减去均值，除以标准差。计算公式为：x* = (x-x.mean)/σ

from sklearn.preprocessing import StandardScaler#一般把train和test集放在一起做标准化，或者在train集上做标准化后，用同样的标准化器去标准化test集scaler = StandardScaler()train = scaler.fit_transform(train)test = scaler.transform(test)

版权声明：本文为CSDN博主「weixin_39602108」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/weixin_39602108/article/details/112022713
'''

In [24]:
# split dataset to train dataset and test dataset.
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test= train_test_split(x1, x2, test_size= 0.2, random_state=0)

In [25]:
#Feature Scaling of datasets  
from sklearn.preprocessing import StandardScaler  
st_x= StandardScaler()  
x_train= st_x.fit_transform(x_train)  
x_test= st_x.transform(x_test)

ValueError: could not convert string to float: '2000-03-07 12:25:20.365709'